In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.insert(0,"../")

In [3]:
# Import

import json

In [4]:
import pandas as pd

share_price_path = "/Users/agrimrustagi/Desktop/Projects/stock-newz/Data/share_prices/20250709_001901.pkl"
financials_path = "/Users/agrimrustagi/Desktop/Projects/stock-newz/Data/stock_financials/20250709_001901.pkl"

dict_share_prices = pd.read_pickle(share_price_path)
dict_financials = pd.read_pickle(financials_path)

In [5]:
quarterly_shareholding_pattern = pd.DataFrame()
yearly_shareholding_pattern = pd.DataFrame()


for stock in dict_financials.keys():
    for analysis_tabs in dict_financials[stock]:
        if analysis_tabs["header"] == "Shareholding Pattern":
            quarterly_table = analysis_tabs["tables"][0]
            quarterly_table["stock"] = stock
            quarterly_shareholding_pattern = pd.concat([quarterly_shareholding_pattern, quarterly_table])

            yearly_table = analysis_tabs["tables"][1]
            yearly_table["stock"] = stock
            yearly_shareholding_pattern = pd.concat([yearly_shareholding_pattern, yearly_table])


In [6]:
quarterly_shareholding_pattern.rename(columns={"Unnamed: 0": "cap_group"}, inplace=True)
yearly_shareholding_pattern.rename(columns={"Unnamed: 0": "cap_group"}, inplace=True)

In [7]:
quarter_fii = quarterly_shareholding_pattern[quarterly_shareholding_pattern["cap_group"] == 'FIIs\xa0+']

In [8]:
quarter_fii.columns

Index(['cap_group', 'Sep 2022', 'Dec 2022', 'Mar 2023', 'Jun 2023', 'Sep 2023',
       'Dec 2023', 'Mar 2024', 'Jun 2024', 'Sep 2024', 'Dec 2024', 'Mar 2025',
       'May 2025', 'stock', 'Jun 2022', 'Apr 2025', 'Jun 2025', 'Jul 2025',
       'Mar 2022'],
      dtype='object')

In [20]:
from src.helper import *

coff_ls = []
stock_ls = []

for stock in quarter_fii.stock.unique():
    coff, df = stock_growth_rate(quarter_fii, stock)
    if coff > 0:
        coff_ls.append(coff)
        stock_ls.append(stock)
        #plot(df, f"{stock} - Growth Rate: {growth_rate:.5f}")


In [ ]:
# Pair each stock with its corresponding growth rate
stock_growth_pairs = list(zip(coff_ls, stock_ls))

# Sort the list in descending order based on growth rate
sorted_pairs = sorted(stock_growth_pairs, key=lambda x: x[0], reverse=True)

# Get the top 20 stocks
top_40 = sorted_pairs[:40]

# Print or extract the top 20 stocks and their growth rates
for growth, stock in top_40:
    print(f"{stock}: {growth:.2f}%")
    coff, df = stock_growth_rate(quarter_fii, stock)
    plot(df, f"{stock} - Growth Rate: {growth:.5f}")
    